In [ ]:
from iduedu import get_boundary
from shapely import LineString

bounds = get_boundary(osm_id=421007)
polygon_coords = " ".join(f"{y} {x}" for x, y in bounds.exterior.coords[:-1])

In [ ]:
from iduedu.modules.downloaders import RequestError
import requests
from iduedu.enums.network_enums import Network
from iduedu import config
import pandas as pd
import numpy as np
from shapely.geometry import LineString
import networkx as nx
from shapely import line_merge, node, MultiLineString
import geopandas as gpd


way_filter = Network.DRIVE.filter

overpass_query = (
    f"""
    [out:json];
        (way{way_filter}(poly:{polygon_coords!r});>;);
    out geom;
    """
)
result = requests.post(config.overpass_url, data={"data": overpass_query}, timeout=config.timeout)
if result.status_code == 200:
    json_result = result.json()["elements"]
else:
    raise RequestError(
        message=f"Request failed with status code {result.status_code}, reason: {result.reason}",
        status_code=result.status_code,
        reason=result.reason,
        response_text=result.text,
        response_content=result.content,
    )

print('downloaded')

data = pd.DataFrame(json_result)
nodes = data[data['type'] == 'node'].copy()
ways = data[data['type'] == 'way'].copy()

ways["geometry"] = [
    LineString([(p["lon"], p["lat"]) for p in line])
    for line in ways["geometry"].values
]

ways["oneway"] = ways["tags"].map(lambda d: d.get("oneway"))


lines = line_merge(node(MultiLineString(ways.geometry.to_list())), directed=True)
lines = gpd.GeoDataFrame(geometry=list(lines.geoms), crs=4326).to_crs(32636)

coords = lines.geometry.get_coordinates().to_numpy()
counts = lines.geometry.count_coordinates()
cuts = np.cumsum(counts)

first_idx = np.r_[0, cuts[:-1]]
last_idx  = cuts - 1

starts = coords[first_idx]
ends   = coords[last_idx]

lines["start"] = list(map(tuple, starts))
lines["end"]   = list(map(tuple, ends))

all_endpoints = pd.Index(lines["start"]).append(pd.Index(lines["end"]))
labels, uniques = pd.factorize(all_endpoints)
n = len(lines)
u = labels[:n]
v = labels[n:]

lines["u"] = u
lines["v"] = v

G = nx.DiGraph()

G.add_nodes_from((i, {"x": float(x), "y": float(y)}) for i, (x, y) in enumerate(uniques))
G.add_edges_from(zip(u,v))


In [ ]:
ways['tags'].apply(pd.Series)

In [ ]:
nx.write_gml(graph, "my_graph.gml")

In [ ]:
from osmnx import graph_from_polygon

graph_osmnx = graph_from_polygon(bounds,network_type='drive')

In [ ]:
nx.write_gml(graph, "my_graph.gml")

In [ ]:

node_ids = set(ways["start"]).union(ways["end"])
needed_nodes = nodes.set_index('id').loc[list(node_ids)].copy()

In [ ]:
from shapely import Point
import geopandas as gpd

needed_nodes['geometry'] = list(map(Point, needed_nodes["lon"], needed_nodes["lat"]))
needed_nodes = gpd.GeoDataFrame(needed_nodes, geometry="geometry", crs=4326)
needed_nodes.explore()

In [ ]:
from networkx import DiGraph

new_graph = DiGraph()



In [ ]:
for _, row in ways.iterrows():



In [ ]:
from networkx import DiGraph

DiGraph().add_edge()

In [ ]:
ways

In [ ]:
gpd.GeoDataFrame(ways, geometry='geometry', crs=4326).explore()

In [ ]:
import geopandas as gpd

gpd.GeoDataFrame(ways, geometry='geometry', crs=4326).explore()

In [ ]:
line_lengths

In [ ]:
ways['geometry'].explode().apply(pd.Series).to_numpy()

In [ ]:
ways['geometry'].apply(len)

In [ ]:
ways['geometry'].apply(pd.Series)

In [ ]:
pd.json_normalize(ways['geometry'].apply(pd.Series))

In [ ]:
nodes[~nodes['tags'].isna()]
